In [1]:
import sys
sys.path.insert(0, '..')

In [1]:
from transformers import pipeline, BitsAndBytesConfig
from PIL import Image    
import torch

2024-05-08 12:25:18.578760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 12:25:18.578783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 12:25:18.579397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 12:25:19.110894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from transformers import pipeline, BitsAndBytesConfig
from PIL import Image    
import torch

nf4_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16
                )
kwargs = {
    "quantization_config": None,
    "trust_remote_code": True,

}
model_id = "microsoft/Phi-3-vision-128k-instruct"


# pipe = pipeline("image-to-text", model=model_id, torch_dtype='auto', device=0, **kwargs)
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/ai2d-demo.jpg"


2024-05-23 01:33:15.664417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 01:33:15.664442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 01:33:15.665078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 01:33:16.199096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_id = "microsoft/Phi-3-vision-128k-instruct" 

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", quantization_config=nf4_config,)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Note
llava is good with diagrams
TinyChart is good with chart

But neither of them should not be quantized, the output is trash. Min is bfloat16

In [3]:
DIRECTORY = 'sample/task1/cambridge/15_1_6'

In [4]:
with open(f'../{DIRECTORY}/answer.txt', 'r') as file:
    student_response = file.read()

with open(f'../{DIRECTORY}/question.txt', 'r') as file:
    essay_topic = file.read()
    
image = Image.open(f'../{DIRECTORY}/image.png')

In [5]:
description = essay_topic.split('\n')[0]

# image = Image.open('IELTS-Writing-Task-1-Bar-Chart-2.jpg')
# description = 'The chart shows the proportion of renewable energy in total energy supply in 4 countries from 1997 to 2010'

In [6]:
messages = [ 
    {"role": "user", "content": f"<|image_1|>\n{description}. Can you provide insightful, detail and precise information about the chart (you can ignore the color)?"}, 
] 
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

In [7]:
generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.2, 
    "do_sample": False, 
}
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [23]:
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response) 

Certainly! The chart is a bar graph that displays the coffee and tea buying and drinking habits of city residents in five Australian cities: Sydney, Melbourne, Brisbane, Adelaide, and Hobart. The vertical axis represents the percentage of city residents, ranging from 0% to 70%, while the horizontal axis lists the five cities.

There are three sets of bars for each city, each representing a different habit:
1. Bought fresh coffee in the last 4 weeks
2. Bought instant coffee in the last 4 weeks
3. Went to a café for coffee or tea in the last 4 weeks

The bars are arranged in a group, with each group representing one city. The height of each bar corresponds to the percentage of city residents who engage in the respective habit.

From the chart, we can observe that the percentage of city residents who bought fresh coffee in the last 4 weeks varies across the cities, with Sydney having the highest percentage and Adelaide the lowest. The percentage of city residents who bought instant coffee

In [5]:
prompt = f"<|user|>\n<image>\n{description}. Extract the features in it.\n<|assistant|>\n"

outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 500})
print(outputs)

NameError: name 'pipe' is not defined

In [5]:
print(outputs[0]['generated_text'])


The diagram below shows the production and processing of milk and dairy production for commercial sale. Extract the features in it
 The diagram shows the production and processing of milk and dairy products for commercial sale. It includes the following features:

1. A milk processing machine that processes milk twice a day.
2. A cheese, cream, and butter production facility that processes milk daily.
3. A truck that transports milk to the processing facility.
4. A truck that transports cheese, cream, and butter from the production facility to the market.
5. A diagram of a dairy farm that shows the production of milk from cows.
6. A diagram of a dairy processing plant that shows the processing of milk into cheese, cream, and butter.
7. A diagram of a dairy market that shows the sale of cheese, cream, and butter to consumers.

The diagram illustrates the entire process of dairy production and processing, from the production of milk on the farm to the sale of dairy products in the marke

The diagram below shows the production and processing of milk and dairy production for commercial sale. Extract the features in it
 The diagram shows the production and processing of milk and dairy products for commercial sale. It includes the following features:

1. A milk processing machine that processes milk twice a day.
2. A cheese, cream, and butter production facility that processes milk daily.
3. A truck that transports milk to the processing facility.
4. A truck that transports cheese, cream, and butter from the production facility to the market.
5. A diagram of a dairy farm that shows the production of milk from cows.
6. A diagram of a dairy processing plant that shows the processing of milk into cheese, cream, and butter.
7. A diagram of a dairy market that shows the sale of cheese, cream, and butter to consumers.

The diagram illustrates the entire process of dairy production and processing, from the production of milk on the farm to the sale of dairy products in the market. It shows how milk is processed into various dairy products, such as cheese, cream, and butter, and how these products are transported to the market for sale. The diagram also shows the role of trucks in transporting milk and dairy products from the production facility to the market.

In [6]:
from tinychart.model.builder import load_pretrained_model
from tinychart.mm_utils import get_model_name_from_path
from tinychart.eval.run_tiny_chart import inference_model
from tinychart.eval.eval_metric import parse_model_output, evaluate_cmds

In [7]:
model_path = "mPLUG/TinyChart-3B-768"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, 
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    device="cuda"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at mPLUG/TinyChart-3B-768 were not used when initializing TinyChartPhiForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.bias', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.m

In [12]:
with torch.no_grad():
    verbal_description = inference_model([f'{DIRECTORY}/image.png'], description + '. Describe it.', model, tokenizer, image_processor, context_len, conv_mode="phi", max_new_tokens=2048)

/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


The chart shows that Sydney has the highest percentage of people who bought fresh coffee in the last 4 weeks at 43%, followed by Melbourne at 41%, and Brisbane at 32%. The data also indicates that Sydney and Melbourne have the highest percentages of people who drank tea in the last 4 weeks at 60% and 61% respectively. In contrast, only 25% of respondents in Hobart had bought tea in the last 4 weeks.


In [13]:
del model
del image_processor
import gc         # garbage collect library
gc.collect()
torch.cuda.empty_cache() 


In [14]:
system_prompt_wt1 ="""
You are an English teaching assistant, and you are good at evaluating essays and reports, and your students need you for their IELTS Academic writing task 1. You will make evaluations based on the topic and the student's response. 
However, since you cannot see the images of charts or diagrams, you will be given a verbal description of them. Sometimes, the description may not be very detailed, or they might contain false information, so just use it as a reference.

You should grade the report's general score and its 4 metrics in IELTS Writing task 1, which are Task Response, Coherence and Cohesion, Lexical Resource and Grammatical Range and Accuracy. 
The overall must be the mean value of 4 metrics' score and can be a float value between 1 and 9 (round to .5), but each metric score must be an integer between 0 and 9.
Recall the IELTS Writing task 1 band score criteria.
The formula to calculate the general score is:
```
General_score = ( ( Task_Response + Coherence_and_Cohesion + Lexical_Resource + Grammatical_Range_and_Accuracy) // 2 ) / 2 
```

In each metric, you should give a detailed explanation and point out exactly the student mistakes that led to that score.
Your output format should be like this:

Here is some tips for you:
- Most of the reports should be written in the third person.
- The report should not have any subjective and personal opinion.
- The report's grade range is mostly around 5.5 to 8.0.
- Every report that has more than 150 words with little spelling or grammatical errors will score at least a 5.5.
- Any attempt of using complex sentences might have at least 4.5 in Grammatical Range and Accuracy.
- The overall score should be the mean value of 4 metric scores, round down to .0 and .5, so make sure your evaluation right.
- Provide constructive feedback.
"""

In [ ]:

verbal_description = outputs[0]['generated_text']

In [15]:
instruction_prompt = f"""
You will have to grade this report in IELTS WRITING task 1 academic guideline
Topic: 
{essay_topic}
Description of the chart or diagram:
{verbal_description}
Report: 
{student_response}
Make sure that you follow the guideline to grade this essay, and do the correct evaluation of general score.
"""

In [26]:
messages = [
    # {"role": "user", "content": system_prompt},
    # {"role": chatbot_role, "content": "Sure"},
    {"role": "system", "content": system_prompt_wt1},
    {"role": "user", "content": instruction_prompt},
]

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.generation_config.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
model_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.6,
    "top_p":0.9,
    "do_sample": True,
}

generation_args = {
    "model_args": model_args,
    "role": 'assistant',
}



pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [28]:
output1 = pipe(messages, **generation_args['model_args'])
output_1_text = output1[0]["generated_text"]
messages.append({"role": "assistant", "content": output_1_text})
print(output_1_text)

Here is the evaluation of the report:

**Task Response: 6**
The report attempts to summarize the main features of the chart, but it lacks clarity and coherence. The introduction is brief and does not provide a clear overview of the chart. The report is divided into three sections, each describing a different aspect of the chart, but the transitions between sections are not smooth. The conclusion is also brief and does not provide a clear summary of the main findings.

The student's mistakes that led to this score are:

* Lack of clear introduction and overview of the chart
* Poor organization and transitions between sections
* Brief and unclear conclusion

**Coherence and Cohesion: 5**
The report has some coherence, but the language is not cohesive. The student uses simple sentences and lacks varied sentence structures. There are also some minor errors in grammar and punctuation. The report could benefit from more connecting words and phrases to improve cohesion.

The student's mistake

In [23]:
from utils.extract import get_score, clean_output, get_essay
from utils.prompt import rescore, indepth_feedback, until_correct, get_instruction_prompt, get_system_prompt

In [29]:
adjust_output  = until_correct(pipe, messages, **generation_args)
general, tr, cc, lr, gr = get_score(adjust_output)

In [31]:
print(clean_output(adjust_output))

**Task Response: 6**
The report attempts to summarize the main features of the chart, but it lacks clarity and coherence. The introduction is brief and does not provide a clear overview of the chart. The report is divided into three sections, each describing a different aspect of the chart, but the transitions between sections are not smooth. The conclusion is also brief and does not provide a clear summary of the main findings.

The student's mistakes that led to this score are:

* Lack of clear introduction and overview of the chart
* Poor organization and transitions between sections
* Brief and unclear conclusion

**Coherence and Cohesion: 5**
The report has some coherence, but the language is not cohesive. The student uses simple sentences and lacks varied sentence structures. There are also some minor errors in grammar and punctuation. The report could benefit from more connecting words and phrases to improve cohesion.

The student's mistakes that led to this score are:

* Simple

In [32]:
messages = [
    # {"role": "user", "content": system_prompt},
    # {"role": chatbot_role, "content": "Sure"},
    {"role": "system", "content": system_prompt_wt1},
    {"role": "user", "content": instruction_prompt},
    {"role": generation_args["role"], "content": clean_output(adjust_output)},
]

In [33]:
rescore_output = rescore(pipe, messages, 'harsh', **generation_args)

In [34]:
print(rescore_output)

I apologize for the harsh evaluation. Upon reevaluation, I will provide a more balanced score. Here is the revised evaluation:

**Task Response: 7**
The report attempts to summarize the main features of the chart, and the student provides some relevant information. While the report could be more detailed and organized, it shows an understanding of the chart's content.

**Coherence and Cohesion: 6**
The report has some coherence, and the student uses simple sentence structures to convey the information. While there are some minor errors in grammar and punctuation, the report is generally easy to follow.

**Lexical Resource: 6**
The report uses a moderate range of vocabulary, and the student repeats some words and phrases. However, the report shows some effort to use precise vocabulary to describe the data.

**Grammatical Range and Accuracy: 6**
The report has some minor errors in grammar and punctuation, but the student uses simple sentence structures and lacks complex sentence structur

In [35]:
rescore_output = until_correct(pipe, messages, **generation_args)
general, tr, cc, lr, gr = get_score(rescore_output)
print(rescore_output)

I apologize for the harsh evaluation. Upon reevaluation, I will provide a more balanced score. Here is the revised evaluation:

**Task Response: 7**
The report attempts to summarize the main features of the chart, and the student provides some relevant information. While the report could be more detailed and organized, it shows an understanding of the chart's content.

**Coherence and Cohesion: 6**
The report has some coherence, and the student uses simple sentence structures to convey the information. While there are some minor errors in grammar and punctuation, the report is generally easy to follow.

**Lexical Resource: 6**
The report uses a moderate range of vocabulary, and the student repeats some words and phrases. However, the report shows some effort to use precise vocabulary to describe the data.

**Grammatical Range and Accuracy: 6**
The report has some minor errors in grammar and punctuation, but the student uses simple sentence structures and lacks complex sentence structur

In [20]:
import re
def get_score(output):
    general = 0
    tr = 0
    cc = 0
    lr = 0
    gr = 0
    match = re.search(r'General\s?:\s?((?<!\d)\d+(?:\.\d+)?)', output)
    if match:
        general = float(match.group(1))
    
    match = re.search(r'General Score\s?:\s?((?<!\d)\d+(?:\.\d+)?)', output)
    if match:
        general = float(match.group(1))    
    
    match = re.search(r'Task Response\s?:\s?((?<!\d)\d+(?:\.\d+)?)', output)
    if match:
        tr = float(match.group(1))
        
    match = re.search(r'Coherence and Cohesion\s?:\s?((?<!\d)\d+(?:\.\d+)?)', output)
    if match:
        cc = float(match.group(1))
        
    match = re.search(r'Lexical Resource\s?:\s?((?<!\d)\d+(?:\.\d+)?)', output)
    if match:
        lr = float(match.group(1))
    
    match = re.search(r'Grammatical Range and Accuracy\s?:\s?((?<!\d)\d+(?:\.\d+)?)', output)
    if match:
        gr = float(match.group(1))
        
    print(general, tr, cc, lr,gr)
    assert general * tr * cc * lr * gr != 0, "Can't extract score from output"
    
    return general, tr, cc, lr, gr

In [21]:
get_score(output_1_text)

6.25 6.0 5.0 6.0 7.0


(6.25, 6.0, 5.0, 6.0, 7.0)